In [1]:
# # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey


### Store Part I results into DataFrame

In [2]:
cities_csv = "Resources/cities.csv"

In [3]:
cities_df = pd.read_csv(cities_csv)
cities_df.dropna(inplace=True)
cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Torbay,47.6666,-52.7314,28.99,93,90,16.11,CA,1612066840
1,Husavik,66.0449,-17.3389,15.80,73,90,8.05,IS,1612067000
2,Norman Wells,65.2820,-126.8329,-23.80,69,75,4.14,CA,1612067000
3,Batagay,67.6333,134.6333,-44.68,77,72,1.41,RU,1612067000
4,Jamestown,42.0970,-79.2353,19.40,73,90,8.05,US,1612067000
...,...,...,...,...,...,...,...,...,...
587,Sol'-Iletsk,51.1631,54.9918,17.10,96,64,5.77,RU,1612067079
588,Gerāsh,27.6652,54.1371,50.11,91,0,3.31,IR,1612067079
589,Cayenne,4.9333,-52.3333,80.60,83,75,11.50,GF,1612066793
590,Tilichiki,60.4667,166.1000,-11.34,90,93,16.71,RU,1612067080


### Humidity Heatmap 

In [4]:
#Configure gmaps.
gmaps.configure(api_key=gkey)

#Use the Lat and Lng as locations and Humidity as the weight.
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"].astype(float)

#plot heatmap
fig = gmaps.figure(zoom_level=2, center=(26.8206, 30.8025))

#Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=15, 
                                 point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [5]:
weather_df = cities_df[cities_df["Max Temp"]<80]
weather_df = weather_df[weather_df["Max Temp"]>70]
weather_df = weather_df[weather_df["Wind Speed"]<10]
weather_df = weather_df[weather_df["Cloudiness"]==0]
weather_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Sur,22.5667,59.5289,71.78,42,0,1.01,OM,1612067000
6,Sanming,26.2486,117.6186,73.58,37,0,1.36,CN,1612067001
131,Pātūr,20.4500,76.9333,72.12,19,0,8.03,IN,1612067017
239,Hong Kong,22.2855,114.1577,73.40,38,0,8.05,HK,1612066922
270,Shwebo,22.5667,95.7000,77.54,32,0,3.38,MM,1612067035
348,Ambilobe,-13.2000,49.0500,75.76,88,0,2.06,MG,1612067047
371,Porto Belo,-27.1578,-48.5531,79.00,94,0,1.14,BR,1612067050
391,Puerto Escondido,15.8500,-97.0667,76.21,65,0,2.64,MX,1612067053
400,Santa Luzia,-19.7697,-43.8514,71.60,60,0,4.61,BR,1612067054
475,Esmeraldas,-19.7625,-44.3139,71.60,68,0,3.44,BR,1612067064


### Hotel Map

In [6]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
weather_df["Hotel Name"] = ""
hotel_df = weather_df
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,Sur,22.5667,59.5289,71.78,42,0,1.01,OM,1612067000,
6,Sanming,26.2486,117.6186,73.58,37,0,1.36,CN,1612067001,
131,Pātūr,20.4500,76.9333,72.12,19,0,8.03,IN,1612067017,
239,Hong Kong,22.2855,114.1577,73.40,38,0,8.05,HK,1612066922,
270,Shwebo,22.5667,95.7000,77.54,32,0,3.38,MM,1612067035,
348,Ambilobe,-13.2000,49.0500,75.76,88,0,2.06,MG,1612067047,
371,Porto Belo,-27.1578,-48.5531,79.00,94,0,1.14,BR,1612067050,
391,Puerto Escondido,15.8500,-97.0667,76.21,65,0,2.64,MX,1612067053,
400,Santa Luzia,-19.7697,-43.8514,71.60,60,0,4.61,BR,1612067054,
475,Esmeraldas,-19.7625,-44.3139,71.60,68,0,3.44,BR,1612067064,


In [ ]:
# # base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "radius": 5000,
#     "types": "lodging",
#     "key": gkey,
#     }
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():

#     # get location from df
#     lat = row["Latitude"]
#     lon = row["Longitude"]
    
#     # change location each iteration while leaving original params in place
#     params["location"] = f"{lat},{lon}"
    
#     base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
#     hotel = requests.get(base_url, params=params)
    
#     hotel = hotel.json()
    
# print(json.dumps(hotel, indent=4, sort_keys=True))


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey,
    }
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    lat = row["Latitude"]
    lon = row["Longitude"]
    city = row["City"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"
    
    response = requests.get(base_url, params=params).json()
    
    #extract results
    results = response["results"]
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
        
    except (KeyError, IndexError):
        print(f"Missing info for {city}")
    print("-----------")

Closest Sur hotel is Sur Plaza Hotel.
-----------
Closest Sanming hotel is Sanming Huangting Lijing Hotel.
-----------
Closest Pātūr hotel is Hotel Rana Washim Road Patur.
-----------
Closest Hong Kong hotel is Mini Hotel Central.
-----------
Closest Shwebo hotel is MOON SHINE Guest House.
-----------
Closest Ambilobe hotel is Hôtel & Restaurant DIANA Ambilobe.
-----------
Closest Porto Belo hotel is Costa Verde Beach Hotel.
-----------
Closest Puerto Escondido hotel is Aldea del Bazar Hotel and Spa.
-----------
Closest Santa Luzia hotel is Pousada Chao Earth.
-----------
Closest Esmeraldas hotel is Bom Será - Marcinho e Fernanda.
-----------
Closest Suleja hotel is Capy Motel.
-----------
Closest Santarém hotel is Hotel Imperador.
-----------
Closest Puerto Suárez hotel is Hotel casa real.
-----------


In [8]:
hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,Sur,22.5667,59.5289,71.78,42,0,1.01,OM,1612067000,Sur Plaza Hotel
6,Sanming,26.2486,117.6186,73.58,37,0,1.36,CN,1612067001,Sanming Huangting Lijing Hotel
131,Pātūr,20.4500,76.9333,72.12,19,0,8.03,IN,1612067017,Hotel Rana Washim Road Patur
239,Hong Kong,22.2855,114.1577,73.40,38,0,8.05,HK,1612066922,Mini Hotel Central
270,Shwebo,22.5667,95.7000,77.54,32,0,3.38,MM,1612067035,MOON SHINE Guest House
348,Ambilobe,-13.2000,49.0500,75.76,88,0,2.06,MG,1612067047,Hôtel & Restaurant DIANA Ambilobe
371,Porto Belo,-27.1578,-48.5531,79.00,94,0,1.14,BR,1612067050,Costa Verde Beach Hotel
391,Puerto Escondido,15.8500,-97.0667,76.21,65,0,2.64,MX,1612067053,Aldea del Bazar Hotel and Spa
400,Santa Luzia,-19.7697,-43.8514,71.60,60,0,4.61,BR,1612067054,Pousada Chao Earth
475,Esmeraldas,-19.7625,-44.3139,71.60,68,0,3.44,BR,1612067064,Bom Será - Marcinho e Fernanda


In [9]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(heat_layer)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
 # Add marker layer ontop of heat map


# Display figure